In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

In [2]:
transactions_basket = pd.read_parquet('transactions.parquet')
clients_basket = pd.read_csv('clients.csv')
materials_basket = pd.read_csv('materials.csv')
plants_basket = pd.read_csv('plants.csv')

In [3]:
def max_number(df_max_number,sum_top_number):
    x_top=0
    max_num=0
    while x_top <= sum_top_number and max_num<=40:
        max_num=max_num+1
        x_top=x_top+df_max_number['sum'].iloc[max_num]
    return max_num

In [4]:
def basket(day_top,sum_top,ta,gender,city,sale,transactions,clients,materials,plants):
    if day_top[0].isalpha():
        transaction_day=transactions.copy()
        transaction_day['weekday'] = transaction_day['chq_date'].dt.weekday_name
        transaction_day=transaction_day[transaction_day['weekday'].isin(day_top)]
    else:
        transaction_day=transactions[transactions['chq_date'].isin(day_top)]
        
    basket_plants = plants.loc[plants['city']==city]
    transaction_day=transaction_day.loc[transaction_day['plant'].isin(basket_plants['plant'].to_list())]
    clients['year']=2020-clients['birthyear']
    basket_clients = clients.loc[(clients['gender']==gender)&(clients['year']>=int(ta.split('-')[0]))&(clients['year']<=int(ta.split('-')[1]))]
    transaction_day=transaction_day.loc[transaction_day['client_id'].isin(basket_clients['client_id'].to_list())]
    
    transaction_day['sum']=transaction_day['sales_sum']/transaction_day['sales_count']
    transaction_day=transaction_day.reset_index(drop=True)
    
    df_basket=pd.DataFrame()
    
    if sale=='Yes':
        df_basket['material']=transaction_day['material'].unique()
        replace_dict = dict(materials[['material','hier_level_4']].to_dict('split')['data'])
        df_basket['hier_level_4'] = df_basket['material'].map(replace_dict)
        transaction_day_i=transaction_day.loc[transaction_day['material'].isin(df_basket['material'].to_list())]
        transaction_day_i=transaction_day_i.reset_index(drop=True)
        df_basket['sum'] = df_basket['material'].map(transaction_day_i.groupby('material')['sum'].mean().to_dict())
        df_basket.sort_values(by=['sum'], inplace=True)
        df_basket=df_basket.reset_index(drop=True)
        df_basket=df_basket.drop_duplicates(subset=['hier_level_4'])
        df_basket=df_basket.reset_index(drop=True)
    else:
        df_basket['material']=transaction_day['material'].value_counts().index
        replace_dict = dict(materials[['material','hier_level_4']].to_dict('split')['data'])
        df_basket['hier_level_4'] = df_basket['material'].map(replace_dict)
        df_basket=df_basket.drop_duplicates(subset=['hier_level_4'])
        df_basket=df_basket.reset_index(drop=True)
    
    transaction_day_i=transaction_day.loc[transaction_day['material'].isin(df_basket['material'].to_list())]
    transaction_day_i=transaction_day_i.reset_index(drop=True)
    df_basket['sum'] = df_basket['material'].map(transaction_day_i.groupby('material')['sum'].mean().to_dict())
    basket_num=max_number(df_basket,sum_top)
    df_basket=df_basket[:basket_num]
    return df_basket

In [5]:
day_basket = 'Saturday, Sunday'
sum_basket = 1000
ta_basket = '18-30'
gender_basket = 'M'
city_basket = 'Moscow'
sale_basket = 'Yes'

In [6]:
data_basket=pd.DataFrame()
data_basket=basket(day_basket.split(', '),sum_basket,ta_basket,gender_basket,city_basket,sale_basket,transactions_basket,clients_basket,materials_basket,plants_basket)
for i in range(len(data_basket)):
    print('material: '+data_basket['material'].iloc[i]+', sum: '+ str(data_basket['sum'].iloc[i].round(2)))

material: 64004ccbb3a438ed6ce7617ff6c72574, sum: 1.0
material: 9f5ad47b8471380dd1b941f08ee12666, sum: 1.0
material: e0b38616dd99a58be17343b4d8acd219, sum: 1.0
material: 89520a5deee68d04b2bb3b801f038807, sum: 1.35
material: 42e60c523e459e862e0d8bed8e27c061, sum: 2.11
material: f146597fc1898fee7318282969b0f0f8, sum: 2.99
material: c0ad282d7bee5534257911d253b57d15, sum: 2.99
material: 363e2efa53532cf576389ce199d99afd, sum: 3.49
material: b75147feae4cf8ec07fb5f2cef855def, sum: 3.56
material: bbe89e3c4c62d83d2ccb8849f8267dfa, sum: 4.89
material: 916fff0c65d2714648ab38e0e7dabb31, sum: 4.99
material: b2392e17a0afee9053fefe7bb6b90353, sum: 5.63
material: 879afb402ee3477a5a9a86f9685021e1, sum: 5.89
material: e246f11eb10a0699fa072cfc3280da43, sum: 6.0
material: 7b47f4e1bf7081b47363949d7742fe02, sum: 6.5
material: 3d7c4c11a9d015625225e2d38a9318ba, sum: 6.52
material: cc34cf21efeee73ac2a689bfb8c2aea7, sum: 6.54
material: e63000bb399c6c0b6e5bc244a2c4b11f, sum: 7.99
material: 0479cc13e597b5f4cb7e576